# Item-to-Item Collaborative Filtering


## Conceito
Seja $u$ o user ativo e $i$ o item alvo.

- Se o user $u$ demonstrou preferência por itens similares ao $i$, é provável que $u$ também goste de $i$.
- Por outro lado, se $u$ não gostou de itens similares ao $i$, é provável que também não goste de $i$.

Em resumo, ao analisar como o user $u$ avaliou itens semelhantes ao $i$, podemos estimar como $u$ avaliaria o item $i$.



## Algoritmo: Filtragem Colaborativa Item-a-Item

O algoritmo de filtragem colaborativa baseada em itens pode ser descrito conforme apresentado em <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a> e <a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">(George Karypis 2001)</a>:

1. **Cálculo das Similaridades entre Itens:**  
    Para cada item do catálogo, identificam-se os $k$ itens mais similares, registrando os valores de similaridade correspondentes. A similaridade entre dois itens pode ser calculada utilizando a *Similaridade Cosseno Ajustada*, que é mais eficaz do que a similaridade cosseno tradicional usada em filtragem colaborativa baseada em users. A fórmula é:

    $$
    w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}
    $$

    Onde $w_{i,j}$ representa o grau de similaridade entre os itens $i$ e $j$, considerando todos os users $u \in U$ que avaliaram ambos os itens. $S^{(i)}$ representa o conjunto dos $k$ itens mais similares ao item $i$.

2. **Geração de Recomendações Top-N para o Usuário:**  
    Para recomendar itens a um usuário $u$ que já interagiu com um conjunto $I_u$ de itens:

    - **Seleção de Candidatos:**  
      Encontra-se o conjunto $C$ de itens candidatos, que é a união dos conjuntos $S^{(i)}$ para todos os itens $i \in I_u$, excluindo os itens já consumidos:

      $$
      C = \bigcup_{i\in I_u}\{S^{(i)}\}\smallsetminus I_u
      $$

    - **Cálculo da Similaridade Total:**  
      Para cada candidato $c \in C$, calcula-se a soma das similaridades entre $c$ e todos os itens do conjunto $I_u$:

      $$
      w_{c,I_u} = \sum_{i\in I_u} w_{c,i}, \forall c \in C
      $$

    - **Ordenação e Seleção dos Itens:**  
      Os itens candidatos são ordenados de acordo com $w_{c,I_u}$ em ordem decrescente, e os $N$ primeiros são recomendados ao usuário.

Este método permite recomendar itens relevantes ao usuário com base nas similaridades entre os itens que ele já avaliou ou consumiu.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, struct, sum as sql_sum, udf
from pyspark.ml.linalg import Vectors
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.ml.feature import Normalizer
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
import time
import math
import os

## Importação das Bibliotecas Necessárias
O código a seguir importa as bibliotecas essenciais para o processamento distribuído de dados, manipulação de DataFrames, criação de vetores esparsos, normalização, clustering (KMeans), e avaliação do modelo. Utilizamos o PySpark para processamento em larga escala, além de pandas para sumarização dos resultados.

In [2]:
start_time = time.time()

## Medição do Tempo de Execução
Inicializamos um temporizador para medir o tempo total de execução do pipeline de recomendação. Isso é útil para avaliar o desempenho do algoritmo em diferentes conjuntos de dados.

In [3]:
spark = SparkSession.builder \
    .appName("ItemItemCF") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set logging level
spark.sparkContext.setLogLevel("WARN")

25/05/31 00:16:18 WARN Utils: Your hostname, cristianonicolau.local resolves to a loopback address: 127.0.0.1; using 192.168.1.122 instead (on interface en0)
25/05/31 00:16:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/31 00:16:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/31 00:16:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Inicialização da SparkSession
Criamos uma sessão Spark, configurando a quantidade de memória e núcleos de execução. A SparkSession é o ponto de entrada para utilizar o PySpark e permite o processamento distribuído dos dados. Também ajustamos o nível de log para evitar excesso de mensagens durante a execução.

In [4]:
df ={
    "small": "./data/ml-latest-small/ratings.csv",
    "ml-1m": "./data/ml-1m/ratings.csv",
    "ml-10m": "./data/ml-10m/ratings.csv",
    "ml-20m": "./data/ml-20m/ratings.csv",
    "ml-25m": "./data/ml-25m/ratings.csv",
}

data_sel = "ml-1m" 

## Definição dos Caminhos dos Datasets (Comentado)
O bloco abaixo define os caminhos para diferentes versões do MovieLens. Está comentado, pois a seleção do dataset pode ser feita de outras formas no código.

In [5]:
# dataset_links = {
#     "small": "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip",
#     "ml-1m": "https://files.grouplens.org/datasets/movielens/ml-1m.zip",
#     "ml-10m": "https://files.grouplens.org/datasets/movielens/ml-10m.zip",
#     "ml-20m": "https://files.grouplens.org/datasets/movielens/ml-20m.zip",
#     "ml-25m": "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
# }

# def download_and_extract_dataset(dataset_name):
#     if dataset_name not in dataset_links:
#         raise ValueError(f"Dataset '{dataset_name}' not found. Available datasets: {list(dataset_links.keys())}")

#     url = dataset_links[dataset_name]
#     os.system(f"wget {url} -O {dataset_name}.zip")
#     os.system(f"unzip {dataset_name}.zip -d {dataset_name}")
#     print(f"Downloaded and extracted {dataset_name} dataset.")

## Download Automático dos Datasets
A função abaixo permite baixar e extrair automaticamente os datasets do MovieLens a partir dos links oficiais. Isso facilita a obtenção dos dados necessários para os experimentos.

In [6]:
PATH = df[data_sel]
# download_and_extract_dataset("ml-20m")  # Change to desired dataset
# PATH = "ml-20m/ml-20m/ratings.csv"  # Adjust path based on the dataset
data = spark.read.csv(PATH, header=True, inferSchema=True) \
            .select("userId", "movieId", "rating")

print("=== DATASET STATISTICS ===")
print("Number of ratings: ", data.count())
print("Average rating: ", data.agg(F.avg("rating")).first()[0])
print("Minimum rating: ", data.agg(F.min("rating")).first()[0])
print("Maximum rating: ", data.agg(F.max("rating")).first()[0])
print("Number of users: ", data.select("userId").distinct().count())
print("Number of movies: ", data.select("movieId").distinct().count())

data.take(5)

=== DATASET STATISTICS ===
Number of ratings:  1000209
Average rating:  3.581564453029317
Minimum rating:  1
Maximum rating:  5
Number of users:  6040
Number of movies:  3706


[Row(userId=1, movieId=1193, rating=5),
 Row(userId=1, movieId=661, rating=3),
 Row(userId=1, movieId=914, rating=3),
 Row(userId=1, movieId=3408, rating=4),
 Row(userId=1, movieId=2355, rating=5)]

## Carregamento e Estatísticas do Dataset
O código a seguir carrega o arquivo de ratings selecionado, exibe estatísticas como número de avaliações, média, mínimo, máximo, número de usuários e de filmes. Isso fornece uma visão geral do conjunto de dados utilizado.

In [7]:
# Split data into training and test sets
ratings, test = data.randomSplit([0.9, 0.1], seed=42)
print(f"Training set size: {ratings.count()}")
print(f"Test set size: {test.count()}")
ratings.cache()
test.cache()

Training set size: 899812
Test set size: 100397


DataFrame[userId: int, movieId: int, rating: int]

## Divisão em Conjuntos de Treino e Teste
Dividimos o dataset em conjuntos de treino (90%) e teste (10%) de forma aleatória. O conjunto de treino é usado para construir o modelo, enquanto o teste serve para avaliar a qualidade das recomendações.

## 3. Model Training: Item-Item CF with LSH
 
The training process involves several steps:
1.  **Create Item Vectors**: For each movie, create a vector where each dimension represents a user, and the value is the rating given by that user. We use sparse vectors for efficiency.
2.  **Normalize Vectors**: Normalize the item vectors using the L2 norm. This makes the LSH approach (based on Euclidean distance) approximate cosine similarity.
3.  **Apply LSH**: Use `BucketedRandomProjectionLSH` to hash the normalized vectors into buckets. Items falling into the same buckets are likely to be similar.
4.  **Find Similar Items**: Use `approxSimilarityJoin` to find pairs of similar items based on their LSH hashes and calculate their cosine similarity.


In [8]:
def to_sparse_vector(user_ratings, size):
    # Sort by userId to ensure strictly increasing indices
    sorted_pairs = sorted(user_ratings, key=lambda x: x.userId)
    indices = [x.userId - 1 for x in sorted_pairs] # Assuming userIds start at 1
    values = [float(x.rating) for x in sorted_pairs]
    return Vectors.sparse(size, indices, values)

### 3.1 Criação dos Vetores dos Itens
A função abaixo transforma as avaliações dos usuários em vetores esparsos, onde cada dimensão representa um usuário e o valor é a nota atribuída ao item. Isso é fundamental para calcular similaridades entre itens de forma eficiente.

now that each rating has been normalized, we can represent each item by a vector of its normalized ratings

In [9]:
spark.udf.register("to_sparse_vector", to_sparse_vector)

item_user = ratings.groupBy("movieId") \
    .agg(collect_list(struct("userId", "rating")).alias("user_ratings"))

### Agrupamento das Avaliações por Item
Registramos a função de conversão para vetor esparso como UDF no Spark e agrupamos as avaliações por `movieId`, preparando os dados para a criação dos vetores de características dos itens.

In [10]:
num_users = ratings.select("userId").distinct().count()

item_vectors_rdd = item_user.rdd.map(
    lambda row: Row(
        movieId=row["movieId"],
        features=to_sparse_vector(row["user_ratings"], num_users)
    )
)
item_vectors = spark.createDataFrame(item_vectors_rdd)
item_vectors.cache()
print("=== ITEM VECTORS ===")
item_vectors.show(5, truncate=False)

=== ITEM VECTORS ===


+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Criação dos Vetores de Características dos Itens
Para cada item, criamos um vetor esparso representando as avaliações dos usuários. Esses vetores serão usados para calcular similaridades entre itens e para o agrupamento via clustering.

### 3.2 Normalize Vectors
 
We normalize the vectors so that Euclidean distance can approximate cosine similarity.


In [11]:
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
normalized_item_vectors = normalizer.transform(item_vectors)
normalized_item_vectors.cache() # Cache for LSH

print("Normalized Item Vectors (sample):")
normalized_item_vectors.show(5, truncate=False)

Normalized Item Vectors (sample):
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 3.2 Normalização dos Vetores
Normalizamos os vetores dos itens utilizando a norma L2. Isso permite que a distância Euclidiana aproxime a similaridade cosseno, facilitando o uso de clustering e cálculo de similaridades.

In [12]:
num_items = item_vectors.count()
k = max(10, min(100, int(math.sqrt(num_items))))
kmeans = KMeans(k=k, seed=42, featuresCol="norm_features", predictionCol="cluster")
kmeans_model = kmeans.fit(normalized_item_vectors)
clustered_items = kmeans_model.transform(normalized_item_vectors).select("movieId", "cluster", "features")


25/05/31 00:16:30 WARN Instrumentation: [0b12aa93] Input vectors will be blockified to blocks, and then cached during training. Be careful of double caching!
25/05/31 00:16:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/31 00:16:33 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:16:33 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB


### 3.3 Agrupamento dos Itens com KMeans
Aplicamos o algoritmo de clustering KMeans sobre os vetores normalizados dos itens. Cada item é atribuído a um cluster, o que permite restringir a busca de vizinhos apenas aos itens do mesmo cluster, tornando o processo mais eficiente.

## 3.3 Apply LSH

We fit an LSH model. The `bucketLength` and `numHashTables` are key parameters:
* `bucketLength`: Affects the width of the buckets. Smaller values lead to more, smaller buckets (higher precision, lower recall).
* `numHashTables`: Increases the chance of finding neighbors by using multiple hash functions (higher recall, more computation).

These parameters often require tuning based on the dataset and desired trade-off.


In [13]:
from pyspark.ml.linalg import DenseVector
from pyspark.sql.types import DoubleType
def cosine_sim(v1, v2):
    return float(v1.dot(v2)) / (float(v1.norm(2)) * float(v2.norm(2)))

cosine_sim_udf = udf(lambda x, y: float(cosine_sim(x, y)), returnType=DoubleType())


### Definição da Similaridade Cosseno
Definimos uma função para calcular a similaridade cosseno entre dois vetores. Essa métrica será utilizada para identificar itens similares dentro de cada cluster.

### 3.4 Find Similar Items
We use `approxSimilarityJoin` to find pairs with a Euclidean distance below a certain `threshold`. We then convert this distance to cosine similarity using the formula: $\cos(\theta) = 1 - \frac{d^2}{2}$. We ensure we have both (i, j) and (j, i) pairs.


In [14]:
cross_joined = clustered_items.alias("a").join(
    clustered_items.alias("b"),
    (col("a.cluster") == col("b.cluster")) & (col("a.movieId") < col("b.movieId"))
)

similarities = cross_joined.withColumn(
    "cosine_sim", cosine_sim_udf(col("a.features"), col("b.features"))
).select(
    col("a.movieId").alias("i_mv"),
    col("b.movieId").alias("j_mv"),
    "cosine_sim"
)

similarities = similarities.union(
    similarities.selectExpr("j_mv as i_mv", "i_mv as j_mv", "cosine_sim")
).cache()


### 3.4 Busca por Itens Similares dentro de Clusters
Realizamos um cross join entre os itens de cada cluster e calculamos a similaridade cosseno entre eles. Apenas pares distintos são considerados. Os pares são duplicados para garantir simetria.

#4. Prediction

To predict the rating for a user *u* on a movie *i*, we use the formula:

$$ \hat{r}_{ui} = \frac{\sum_{j \in N(i)} s_{ij} \cdot r_{uj}}{\sum_{j \in N(i)} |s_{ij}|} $$

Where:
* $N(i)$ is the set of neighbors of movie *i*.
* $s_{ij}$ is the similarity between movie *i* and movie *j*.
* $r_{uj}$ is the rating given by user *u* to movie *j*.

We achieve this by joining the `test_data` with `similarities` and then with `train_data`.


In [15]:
# Join test data with similarities to find neighbors for target movies
test_neighbors = test.alias("t") \
    .join(similarities.alias("s"), col("t.movieId") == col("s.i_mv"))

# Join with train data to get user ratings for neighbor movies
test_with_ratings = test_neighbors \
    .join(ratings.alias("r"), (col("t.userId") == col("r.userId")) & (col("s.j_mv") == col("r.movieId"))) \
    .select(
        col("t.userId"),
        col("t.movieId").alias("target_movie"),
        col("s.j_mv").alias("neighbor_movie"),
        col("s.cosine_sim"),
        col("r.rating").alias("neighbor_rating")
    )

print("Test Data with Neighbor Ratings (sample):")
test_with_ratings.show(5, truncate=False)

Test Data with Neighbor Ratings (sample):


25/05/31 00:16:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:16:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:16:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:16:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:16:35 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:17:19 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:17:22 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


+------+------------+--------------+-------------------+---------------+
|userId|target_movie|neighbor_movie|cosine_sim         |neighbor_rating|
+------+------------+--------------+-------------------+---------------+
|1     |588         |531           |0.25318648459870136|4              |
|1     |595         |531           |0.2682293510466324 |4              |
|1     |588         |594           |0.45936688757507194|4              |
|1     |595         |594           |0.5126766114118322 |4              |
|1     |588         |1566          |0.42012325590165744|4              |
+------+------------+--------------+-------------------+---------------+
only showing top 5 rows



## 4. Predição das Avaliações
Para prever a nota que um usuário daria a um filme, buscamos os vizinhos do item alvo e as avaliações do usuário nesses vizinhos. O código a seguir realiza os joins necessários para obter essas informações.

Now, we calculate the weighted average. We add a small epsilon or use `F.when` to avoid division by zero if the sum of similarities is zero.


In [16]:
# Calculate weighted sum and sum of weights
weighted_sums = test_with_ratings.groupBy("userId", "target_movie").agg(
    sql_sum(col("cosine_sim") * col("neighbor_rating")).alias("weighted_rating_sum"),
    sql_sum(F.abs(col("cosine_sim"))).alias("similarity_sum") # Use absolute for the denominator
)

# Predict ratings, handling potential division by zero
predictions = weighted_sums.withColumn(
    "pred_rating",
    F.when(
        col("similarity_sum") > 0,
        col("weighted_rating_sum") / col("similarity_sum")
    ).otherwise(None) 
).filter(col("pred_rating").isNotNull()) \
 .select("userId", "target_movie", "pred_rating")

print("Predicted Ratings (sample):")
predictions.show(5, truncate=False)


Predicted Ratings (sample):


25/05/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:17:25 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/31 00:17:29 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


+------+------------+------------------+
|userId|target_movie|pred_rating       |
+------+------------+------------------+
|26    |2403        |2.948758554673438 |
|58    |32          |3.964580125015903 |
|97    |1248        |4.667354193446279 |
|123   |2502        |2.726767936516566 |
|131   |628         |3.4406109335616875|
+------+------------+------------------+
only showing top 5 rows



### Cálculo da Média Ponderada das Avaliações
Calculamos a média ponderada das avaliações dos vizinhos, utilizando as similaridades como pesos. Isso resulta na predição da nota para cada usuário-filme no conjunto de teste.

We evaluate the predictions against the actual ratings in the `test_data` using **Root Mean Squared Error (RMSE)** and **Mean Absolute Error (MAE)**.


In [17]:
# Join predictions with actual ratings from the test set
final_results = predictions.alias("p") \
    .join(test.alias("t"), (col("p.userId") == col("t.userId")) & (col("p.target_movie") == col("t.movieId"))) \
    .select(
        col("p.userId"),
        col("p.target_movie"),
        col("p.pred_rating"),
        col("t.rating").alias("actual_rating")
    )

# Ensure no null predictions are passed to the evaluator
final_results_filtered = final_results.filter(col("pred_rating").isNotNull())
final_results_filtered.cache()

print("Final Results with Predictions and Actuals (sample):")
final_results_filtered.show(10, truncate=False)


Final Results with Predictions and Actuals (sample):


25/05/31 00:17:30 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/05/31 00:17:32 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/31 00:17:35 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


+------+------------+------------------+-------------+
|userId|target_movie|pred_rating       |actual_rating|
+------+------------+------------------+-------------+
|26    |2403        |2.948758554673438 |3            |
|58    |32          |3.964580125015903 |5            |
|97    |1248        |4.667354193446279 |5            |
|123   |2502        |2.726767936516566 |3            |
|131   |628         |3.4406109335616875|3            |
|148   |1894        |3.49470001270962  |4            |
|166   |1297        |3.8032702823273605|3            |
|187   |1057        |4.348987812110765 |5            |
|195   |1573        |3.9175901147874392|3            |
|219   |480         |3.7172059356197384|3            |
+------+------------+------------------+-------------+
only showing top 10 rows



25/05/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


## Avaliação das Predições
Comparamos as predições com as avaliações reais do conjunto de teste utilizando as métricas RMSE (Root Mean Squared Error) e MAE (Mean Absolute Error), que quantificam o erro das recomendações.

In [18]:
# calculate RMSE
evaluator = RegressionEvaluator(
    labelCol="actual_rating",
    predictionCol="pred_rating",
    metricName="rmse"
)
rmse = evaluator.evaluate(final_results_filtered)
print(f"Root Mean Squared Error (RMSE): {rmse}")

#calculate MAE
mae_evaluator = RegressionEvaluator(
    labelCol="actual_rating",
    predictionCol="pred_rating",
    metricName="mae"
)
mae = mae_evaluator.evaluate(final_results_filtered)
print(f"Mean Absolute Error (MAE): {mae}")

25/05/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


Root Mean Squared Error (RMSE): 0.9614173994293513
Mean Absolute Error (MAE): 0.7502739194296247


25/05/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


In [ ]:
# Tempo total de execução
end_time = time.time()
execution_time = round(end_time - start_time, 2)

summary = pd.DataFrame([{
    'Dataset': data_sel,
    'Train Size': ratings.count(),
    'Test Size': test.count(),
    'Similarity': 'Cosine (within cluster)',
    'Clustering': f"KMeans (k={k})",
    'RMSE': rmse,
    'MAE': mae,
    'Execution Time (s)': round(time.time() - start_time, 2)
}])


# Salvar o resumo
output_path = './output/item_item_cf_summary.csv'
if os.path.exists(output_path):
    summary.to_csv(output_path, mode='a', header=False, index=False)
else:
    summary.to_csv(output_path, index=False)

print("Resultados salvos em './output/'")


25/05/31 00:17:37 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


Resultados salvos em './output/'


## Resumo e Salvamento dos Resultados
O código abaixo resume as principais métricas do experimento (tamanho dos conjuntos, parâmetros, erros e tempo de execução) e salva os resultados em um arquivo CSV para análise posterior.

In [20]:
spark.stop()

## Encerramento da Sessão Spark
Por fim, encerramos a sessão Spark para liberar os recursos computacionais utilizados durante o processamento.